In [2]:
# -- Pre-run: change to parent directory for importing library -- only run this once!
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
os.chdir(module_path)

In [3]:
# -- Notebook start
import folium
import logging

from datakit import Itinerum
import datakit_config

logging.getLogger().setLevel(logging.WARNING)

In [4]:
# initialize Itinerum library with config
itinerum = Itinerum(config=datakit_config)

# read .csvs defined in config if not already available
itinerum.setup(force=False)

In [5]:
# load all users and find a real one with many coordinates recorded by ID
users = itinerum.load_all_users()

test_user = [u for u in users if str(u.uuid).startswith('563fb')][0]

In [6]:
# run trip detection on user with config parameters
parameters = {
    'subway_stations': itinerum.database.load_subway_entrances(),
    'break_interval_seconds': 120,
    'subway_buffer_meters': datakit_config.TRIP_DETECTION_SUBWAY_BUFFER_METERS,
    'cold_start_distance': datakit_config.TRIP_DETECTION_COLD_START_DISTANCE_METERS,
    'accuracy_cutoff_meters': datakit_config.TRIP_DETECTION_ACCURACY_CUTOFF_METERS    
}
results = itinerum.run_process(itinerum.process.trip_detection.triplab.algorithm, [test_user], parameters)

In [7]:
# save detected trips for user
trips, summaries = results[test_user]
trip_points = []
for trip in trips.values():
    for point in trip:
        point['uuid'] = test_user.uuid
        trip_points.append(point)
itinerum.database.save_trips(trip_points)

# reload user to build trip objects (this should be done automatically by trip detection,
# perhaps there should be a wrapper in the .trip_detection module to perform this on all add algorithms)
test_user = itinerum.database.load_user(uuid=test_user.uuid)

# examine the user's trips
test_user.trips

{1: <Trip num=1 code=1>,
 2: <Trip num=2 code=1>,
 3: <Trip num=3 code=1>,
 4: <Trip num=4 code=1>,
 5: <Trip num=5 code=1>,
 6: <Trip num=6 code=202>,
 7: <Trip num=7 code=202>,
 8: <Trip num=8 code=202>,
 9: <Trip num=9 code=1>,
 10: <Trip num=10 code=202>,
 11: <Trip num=11 code=202>,
 12: <Trip num=12 code=202>,
 13: <Trip num=13 code=202>,
 14: <Trip num=14 code=202>,
 15: <Trip num=15 code=202>,
 16: <Trip num=16 code=202>,
 17: <Trip num=17 code=1>,
 18: <Trip num=18 code=1>,
 19: <Trip num=19 code=202>,
 20: <Trip num=20 code=1>}

In [8]:
# examine a trip's attributes
test_trip = test_user.trips[9]
print('distance:', test_trip.distance)
print('start:', test_trip.start_UTC, 'end:', test_trip.end_UTC)
print('code:', test_trip.trip_code)
test_trip

distance: 6997.220991316412
start: 2018-03-28 16:27:55.277000 end: 2018-03-28 16:53:34.403000
code: 1


<Trip num=9 code=1>

In [9]:
# plot trip on map
m = folium.Map()
coordinates = [(p.latitude, p.longitude) for p in test_trip.points]
polyline = folium.PolyLine(locations=coordinates).add_to(m)
m.fit_bounds(coordinates)
m

In [10]:
# attempt to map match between various mode networks using OSRM and compare the match quality
itinerum.process.map_match.osrm.match(coordinates=test_trip.points, matcher='BIKING')

ConnectionError: HTTPSConnectionPool(host='osrmserver.com', port=443): Max retries exceeded with url: /osrm/match/v1/biking/-73.6012288,45.5397842;-73.6017414,45.5378039;-73.6016891,45.5378723;-73.6016531,45.5379354;-73.6017262,45.5377109;-73.6018874,45.5375295;-73.6021461,45.5373126;-73.6022918,45.5371204;-73.602472,45.5369015;-73.6026336,45.5367336;-73.6027581,45.5365634;-73.6028802,45.536395;-73.6029906,45.5362431;-73.6030512,45.5362247;-73.6032223,45.5360651;-73.6033717,45.5358684;-73.6035024,45.5357339;-73.6036164,45.5356255;-73.6036951,45.5355578;-73.603844,45.5354148;-73.6039673,45.5352414;-73.6041595,45.5350381;-73.6043805,45.5348317;-73.6045671,45.5345972;-73.6048221,45.5344095;-73.6050222,45.5341813;-73.6052111,45.5339461;-73.6053679,45.5336949;-73.6055829,45.5334626;-73.6057935,45.533284;-73.6058688,45.5332297;-73.6059893,45.533128;-73.6060874,45.5329789;-73.6061733,45.5328025;-73.6063451,45.5325997;-73.6065213,45.5324138;-73.6066632,45.5322568;-73.6068052,45.5320911;-73.606

In [20]:
# write geojson features
## write user coordinates to geojson as points
itinerum.io.write_input_geojson(cfg=datakit_config,
                                fn_base=str(test_user.uuid),
                                coordinates=test_user.coordinates,
                                prompts=test_user.prompt_responses,
                                cancelled_prompts=test_user.cancelled_prompt_responses)

## write user detected trip points and links as linestrings to geojson
itinerum.io.write_trips_geojson(cfg=datakit_config, fn_base=str(test_user.uuid), trips=test_user.trips)